In [2]:
from matplotlib.backends.backend_pdf import PdfPages
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#%matplotlib notebook
%matplotlib inline
import operator
import time
import ast
import scipy
import scipy.stats
import seaborn as sns
import numpy as np
import powerlaw
from collections import defaultdict
from srm.db_manager import connect_to_db, execute_sql

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [85]:
db_name, table = "bidisha", "copaWordCup"
cursor_mysql, conn = connect_to_db("localhost", "root", "root")

In [3]:
def plotData(data, start, end):
    count = 0
    bins = np.linspace(start, end, 10)
    colors = cm.rainbow(np.linspace(0, 1, len(data.keys())))
    #print colors
    #colors = ['r', 'b', 'g', 'w', 'y', 'o', ''
    
    plt.figure(figsize=(20,20))
    moddata=defaultdict(list)
    
    for k in data.keys():
        timestamp=data[k]
        for time in timestamp:
            if time>=start and time<=end :
                moddata.setdefault(k,[]).append(time)
    
    for (c,k) in zip(colors, data):
        #print k
        #k = unicode(k, "utf-8")
        count+=1
        plt.hist(moddata[k], bins, label = k, histtype='step', color = c)
    plt.yscale('log')
    plt.legend(loc='upper right', prop={'size':10})
    plt.xlabel('Tweeted time', fontsize=18)
    plt.ylabel('Tweet volume', fontsize=18)
    plt.show()

In [4]:
def getuniquehashtag():
    results = execute_sql("Select distinct(hashtags) from %s.%s", (db_name, table))
    #hashtagMod1, hashtagMod2, hashtagMod3))
    hashtagName = []               
    for row in results:
            hashtagName.extend(row[0].split(','))
    return list(set(hashtagName))

In [5]:
def relativetimestamp(hashtags):
    dicthashtag = defaultdict(list)
    mintimestamp = [row[0] for row in execute_sql('Select min(created_at) from %s.%s',(db_name, table))]
    mintimestamp = int(time.mktime(mintimestamp[0].timetuple()))
    for hashtag in hashtags:
        hashtagMod1 = '\"%'+hashtag.strip()+',%\"'
        hashtagMod2 = '\"%,'+hashtag.strip()+'%\"'
        hashtagMod3 = '\"'+hashtag.strip()+'\"'
        timestamps = [row[0] for row in execute_sql("Select created_at from %s.%s where\
        (hashTags like %s OR hashTags like %s OR hashTags like %s) \
        order by created_at;", (db_name, table, hashtagMod1, hashtagMod2, hashtagMod3))]
        relativetimestamps = [(int(time.mktime(x.timetuple())) - mintimestamp) for x in timestamps]
        tempstr=','.join(str(x) for x in relativetimestamps)
        dicthashtag[hashtag] = relativetimestamps
    return dicthashtag

In [20]:
def generatehtFromFile(filename):
    f = open(filename)
    hashtaglist = []
    for line in f:
        hashtaglist.append(line.split("\t")[0])
    return hashtaglist

In [7]:
def timestamp(hashtags, c):
    dicthashtag = defaultdict(list)
    for hashtag in hashtags:
        timestamps = [row[0] for row in c.execute('Select UnixTimeStamp from hashtagUsageInfo where Hashtag=\\
    '+'\"'+hashtag+'\"'+'order by UnixTimeStamp')]
        relativetimestamps = [(x - timestamps[0]) for x in timestamps]
        tempstr=','.join(str(x) for x in relativetimestamps)
        dicthashtag[hashtag] = relativetimestamps
    return dicthashtag

In [44]:
def generateDataFromFile(filename):
    f = open(filename)
    dicthashtag = defaultdict(list)
    for line in f:
        tokens = line.strip().split("\t")
        hashtag = tokens[0]
        #print hashtag
        timestamp = ast.literal_eval(tokens[1])
        dicthashtag[hashtag] = timestamp
    return dicthashtag

In [35]:
def calculateRank(dicthashtag, starttime, endtime, k):
    popularity_count = defaultdict(int)
    for key in dicthashtag.keys():
        timestamplist = dicthashtag[key]
        #print key,timestamplist
        for ts in timestamplist:
            if(ts>=starttime and ts<=endtime):
                popularity_count.setdefault(key, 0)
                popularity_count[key]+=1
    #print popularity_count
    sortedpc = sorted(popularity_count.items(), key=operator.itemgetter(1), reverse=True)
    #sortedpc = sorted(popularity_count, key=popularity_count.get, reverse=True)
    #print sortedpc
    topk = [row[0] for row in sortedpc[:k]]
    print topk
    resultDict = defaultdict(int)
    for tk in topk:
        #print defaultdict[tk]
        resultDict[tk] = dicthashtag[tk]
    return resultDict
    

In [86]:
hashtagList = getuniquehashtag()
hashtagmap = relativetimestamp(hashtagList)
f = open("/home/bidisha/2017-hashtag-code/Data/"+table+"relative.txt","a")
countmap = defaultdict(list)
for hashtag in hashtagmap.keys():
    timestamps = hashtagmap[hashtag]
    if len(timestamps) > 200:
        countmap[hashtag] = len(timestamps)
        f.write(hashtag+"\t")
f.close()
f = open("/home/bidisha/2017-hashtag-code/Data/"+table+".txt","a")
for hashtag in countmap.keys():
    f.write(hashtag+"\t"+str(countmap[hashtag])+"\n")
    #print hashtag, countmap[hashtag]
f.close()

In [63]:
hashtagdict = generateDataFromFile("/home/bidisha/2017-hashtag-code/Data/BaselineDisaster_Nepalrelative.txt")
hashtagList = generatehtFromFile("/home/bidisha/2017-hashtag-code/Data/hashtags/Disaster_Nepal_mini.txt")

#hashtagdict = generateDataFromFile("/home/bidisha/2017-hashtag-code/Data/Hillaryrelative.txt")
#hashtagList = generatehtFromFile("/home/bidisha/2017-hashtag-code/Data/hashtags/Hillary.txt")

newhtdict = defaultdict(list)
for key in hashtagList:
    key = key.strip()
    newhtdict[key] = hashtagdict[key]

#print newhtdict
#plotData(newhtdict, 0,1296000)
#plotData(newhtdict, 432000,1296000)

In [65]:
tempdict = calculateRank(newhtdict, 0, 1296000, 30)
#print tempdict
tempdict1 = calculateRank(newhtdict, 0, 86400, 30)
tempdict2 = calculateRank(newhtdict, 86400, 162800, 30)

#tempdict1 = calculateRank(newhtdict, 345600, 691200, 100)
#tempdict2 = calculateRank(newhtdict, 518400,777600, 100)
print (set(tempdict1).intersection(tempdict2))
print set(tempdict2).intersection(tempdict3)
#tempdict = 

['nepalquake', 'nepalquak', 'prayfornepal', 'earth', 'pray', 'prayfor', 'kathmandu', 'nepalquakerelief', 'indiawithnepal', 'NepalDisasterReliefByMSG', 'withnepal', 'NepalDisaster', 'earthquakeagain', 'nepaldisasterrelief', 'nepalquakeagain', 'quakerelief', 'langtang', 'MSGHelpEarthquakeVictims', 'Soulvulture', 'prayer', 'victims', 'quakeagain', 'terremoto', 'earthquakevictims', 'redcross', 'terremotonepal', 'helpnepal', 'mounteverest', 'staystrong', 'bangladesh']
['nepalquake', 'prayfornepal', 'nepalquak', 'earth', 'pray', 'prayfor', 'kathmandu', 'nepalquakerelief', 'earthquakeagain', 'nepalquakeagain', 'indiawithnepal', 'withnepal', 'quakeagain', 'Soulvulture', 'prayer', 'NepalDisasterReliefByMSG', 'quakerelief', 'redcross', 'langtang', 'mounteverest', 'terremoto', 'prayersfornepal', 'staystrong', 'MSGHelpEarthquakeVictims', 'terremotonepal', 'bangladesh', 'nepalquake2015', 'helpnepal', 'kathmanduquake', 'victims']
['nepalquake', 'nepalquak', 'prayfornepal', 'pray', 'earth', 'kathmand